In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import csv
from os import listdir

In [ ]:
# -------- range of the variables ----------
V_S = 7.0                   # service speed [kn]
range_D     = [0.5, 0.8]
range_AEdAO = [0.3, 1.05]
range_PdD   = [0.5, 1.4]
range_Z     = [2, 7]

# Define the lower and upper bounds for each variable
lower_bounds = [range_D[0], range_AEdAO[0], range_PdD[0]]
upper_bounds = [range_D[1], range_AEdAO[1], range_PdD[1]]

## Load files

In [ ]:
# path to directory containing all the solvers folders 
dir_run = '../Main/run'

In [ ]:
def get_best_result_df(best_result_file_path):
    history = []
    with open(best_result_file_path, 'r', newline='') as file:
        reader = csv.reader(file)
        counter = 0
        for row in reader:
            if counter == 1:
                params = row
            elif counter == 3:
                history = row
            elif counter == 5:
                configs = row
            counter += 1
    # D, AEdAO, PdD, Z, fitness
    D       = float(params[0])
    AEdAO   = float(params[1])
    PdD     = float(params[2])
    Z       = int(params[3]) if len(params[3]) == 1 else int(float(params[3]))
    fitness = float(params[6])
    # history    
    if len(history) > 0:
        history = [float(h) for h in history]
    # configs
    solver_name = configs[0]
    vs        = float(configs[1])
    n_pop       = int(configs[2])
    max_it      = int(configs[3])
    seed        = int(configs[4])
    # create new entry in df
    new_data = {'D': D, 
                'AEdAO': AEdAO, 
                'PdD': PdD,
                'Z': int(Z),
                'Brake Power': -fitness,
                'Seed': seed,
                'Algorithm': solver_name,
                'VS': vs,
                'Npop': n_pop,
                'maxIt': max_it}
    return new_data

In [ ]:
data = {'D': [], 'AEdAO': [], 'PdD': [], 'Z': [], 'Brake Power': [], 'Seed': [], 'Algorithm': [], 'VS': [], 'Npop':[], 'maxIt':[]}
df_results = pd.DataFrame(data)

# enter in the folders until reach the files with the executions
for solver_run in listdir(dir_run):
    solver_run = dir_run+'/'+solver_run
    # dir's created by the multiple_run_solver
    for solver_vs in listdir(solver_run):
        solver_vs = solver_run+'/'+solver_vs
        for dir_seed in listdir(solver_vs):
            dir_seed = solver_vs+'/'+dir_seed
            # get the file with best results for that seed execution
            best_result_file = [filename for filename in listdir(dir_seed) if 'best_results' in filename]
            if len(best_result_file) > 0:
                best_result_file = best_result_file[0]
                best_result_file = dir_seed+'/'+best_result_file
                new_data = get_best_result_df(best_result_file)
                df_results = pd.concat([df_results, pd.DataFrame(new_data, index=[0])], ignore_index=True)
            else:
                print('not able to find best result file in', dir_seed)


In [ ]:
# remove not used 'pop size', 'max generations'
df_results = df_results.drop(['Npop', 'maxIt'], axis=1)

In [ ]:
# rename Algorithm col
solver_renames = {'cmaes':    'CMA-ES',
                  'openaies': 'OpenAI-ES',
                  'DE':       'DE Replicated',
                  'DE_mod':   'DE Proposed'
                 }

df_results['Algorithm'] = df_results['Algorithm'].replace(solver_renames)

In [ ]:
len(df_results)

In [ ]:
df_results

show info

In [ ]:
df_results.groupby(['VS', 'Algorithm'])['Brake Power'].mean()

Compare

In [ ]:
# Find the index of the row with the minimal fitness value
min_fitness_index = df_results.groupby(['VS'])['Brake Power'].idxmin()

# Get the row with the minimal fitness value
min_fitness_row = df_results.loc[min_fitness_index]

min_fitness_row

In [ ]:
lower_values = df_results.groupby(['VS', 'Algorithm'])['Brake Power'].min()
print(lower_values)

In [ ]:
df_vs_7_0 = df_results[df_results['VS'] == 7]
df_vs_7_5 = df_results[df_results['VS'] == 7.5]
df_vs_8_0 = df_results[df_results['VS'] == 8]
df_vs_8_5 = df_results[df_results['VS'] == 8.5]

## Tables

In [ ]:
def get_min_row(data):
    index_min_PB = data.groupby('Algorithm')['Brake Power'].idxmin()
    # Select original rows using index
    rows_min_PB = data.loc[index_min_PB]
    return rows_min_PB

def get_min(data):
    min_PB = data.groupby('Algorithm')['Brake Power'].min()
    return min_PB

In [ ]:
vs = 8.0

In [ ]:
df_results[df_results['VS'] == vs].groupby('Algorithm')['Brake Power'].min()

In [ ]:
print('VS Algortihm D AE/AO P/D Z')
for r in get_min_row(data = df_vs_8_5).to_numpy():
    d    = r[0]
    aedao= r[1]
    pdd  = r[2]
    z    = r[3]
    pb   = r[4]
    algor= r[5]
    vs   = r[6]
    z = int(z)
    print(f'{vs} & {algor} & {d:.3f} & {aedao:.3f} & {pdd:.3f} & {z}' + " \\\\" )

In [ ]:
df_results[(df_results['VS'] == 8.5) & (df_results['Algorithm'] == 'DE Proposed')]

## Grafics

In [ ]:
def create_img(data, order, save=False, name='img'):
    fig, ax = plt.subplots()

    sns.boxplot(y="Brake Power",
                x = "Algorithm",
                hue = 'Algorithm',
                data=data,
                order=order
                )

    sns.set_style("whitegrid")
    sns.set(rc={'figure.figsize':(12,9)})
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=15)
    title_text = "VS = "
    for v in data['VS'].unique():
        title_text += str(v)
    plt.title(title_text, fontsize=15)
    plt.xlabel('Algorithm',  fontsize=16)
    plt.ylabel('Brake Power',fontsize=18)

    if save:
        plt.tight_layout()
        plt.savefig(name+".pdf", dpi=300)

In [ ]:
save = False

In [ ]:
create_img(data = df_vs_7_0,
           order=['DE Replicated', 'DE Proposed', 'CMA-ES', 'OpenAI-ES'],
           save=save,
           name='boxplot_7_0' )

In [ ]:
create_img(data = df_vs_7_5,
           order=['DE Replicated', 'DE Proposed', 'CMA-ES', 'OpenAI-ES'],
           save=save,
           name='boxplot_7_5' )

In [ ]:
create_img(data = df_vs_8_0,
           order=['DE Proposed', 'CMA-ES', 'OpenAI-ES'],
           save=save,
           name='boxplot_8_0' )

In [ ]:
create_img(data = df_vs_8_5,
           order=['DE Proposed', 'CMA-ES', 'OpenAI-ES'],
           save=save,
           name='boxplot_8_5' )

## Double graphs

In [ ]:
rename_dict = {'CMA-ES': 'CMA-ES',
               'OpenAI-ES': 'OpenAI-ES',
               'DE Replicated': 'DE\nReplicated',
               'DE Proposed': 'DE\nProposed'
              }

custom_order = {
    'DE\nReplicated': 1,
    'DE\nProposed':   2,
    'CMA-ES':         3,
    'OpenAI-ES':      4
}

In [ ]:
font_size = 14

def multiple_boxplot(vs_values, save=False, only_put_1_legend=False):
    # Create subplots with a shared x-axis
    fig, ax = plt.subplots(1, len(vs_values), sharey='col', figsize=(11, 7))

    for i, vs in enumerate(vs_values):
        df_temp = df_results[df_results['VS'] == vs]
        # rename 
        df_temp.loc[:,'Algorithm'] = df_temp['Algorithm'].map(rename_dict)
        # Get unique values and sort them based on the custom order
        unique_values = sorted(df_temp['Algorithm'].unique(), key=lambda x: custom_order[x])

        # create the box plot
        sns.boxplot(y="Brake Power", 
                    x = "Algorithm",
                    hue = 'Algorithm',
                    order=unique_values,
#                         palette="hls", # Colors
                    data=df_temp, ax=ax[i])  

        # Set the title for the subplot
        ax[i].set_title(f'VS = {vs}',   fontsize=font_size)
        ax[i].tick_params(axis='both', labelsize=font_size) 
        # Set labels for x and y axes
        ax[i].set_xlabel('Algorithm',   fontsize=font_size)
        ax[i].set_ylabel('Brake Power', fontsize=font_size)

        # ylim
        minmax = df_temp['Brake Power'].agg(['min', 'max']).to_list()
        ax[i].set_ylim(minmax[0]*0.999, minmax[1]*1.001) 
        
        if only_put_1_legend:
            # only put 1 legend
            if i == 1:
        #         ax[i].legend(loc='upper right')
                legend = ax[i].legend(loc='upper left', bbox_to_anchor=(1, 1))
            else:
                ax[i].legend([],[], frameon=False)

    # Adjust spacing between subplots
    plt.tight_layout()
    
    # save
    if save:
        name_f  = 'boxplot'
        # speed
        name_f += '_' + "and".join([str(v) for v in vs_values])
        name_f  = name_f.replace('.','_')
        # file type
        name_f += '.pdf'
        plt.savefig(name_f, bbox_inches='tight')
        print('saved', name_f)

    # Show the plot
    plt.show()

In [ ]:
save=True

In [ ]:
multiple_boxplot([7.0, 7.5], save=save)

In [ ]:
multiple_boxplot([8.0, 8.5], save=save)